### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [1]:
train, dev, test = [], [], []

In [2]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
# print (len(train))
# print (train[:3])

In [3]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
# print (len(dev))
# print (dev[:3])

#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [4]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
# print (len(test))
# print (test[:3])

#### You can split every sentence into lists of words by white spaces.

In [5]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
train_split = train_split + dev_split
test_split = [x.split(' ') for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [6]:
# import required library
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from keras.models import Sequential
from collections import defaultdict
from tensorflow.keras import optimizers
from keras.initializers import Constant
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers import LSTM, Dropout, Dense, Flatten, Bidirectional



In [7]:
# train using Word2Vec to create vector embedding
train_sentences = [item[1] for item in train_split]
maxList = max(train_sentences, key = lambda i: len(i))
max_sentence_length = len(maxList)

word2Vec_model = Word2Vec(sentences = train_sentences, vector_size = 100, window = 10, min_count = 1, sg = 1)
word2Vec_weights = word2Vec_model.wv.vectors
word2Vec_vocab_size, word2Vec_embedding_size = word2Vec_weights.shape
vocabulary = list(word2Vec_model.wv.index_to_key)

word2Vec_dict = {}
for word in vocabulary:
    word2Vec_dict[word] = word2Vec_model.wv.get_vector(word)

In [8]:
# data prepocessing
X_test =  test_split
X_train = [item[1] for item in train_split]
Y_train = [item[0] for item in train_split]

train_dict = defaultdict(int)
for i, sentence in enumerate(X_train):
    for word in sentence:
        train_dict[word] += 1

# print(len(train_dict))
# print(len(vocabulary))

min_freq, max_freq = 1, 8000
train_dict = {k:v for k, v in train_dict.items() if v>=min_freq and v<=max_freq}
rank_words = {key: rank for rank, key in enumerate(sorted(train_dict, key=train_dict.get, reverse=True), 1)}
vocabulary_size = len(rank_words)

In [9]:
# create mbedding matrix
embedding_matrix = np.zeros(shape=(vocabulary_size + 1, word2Vec_embedding_size))
for i,word in enumerate(rank_words):
  embedding_vector = word2Vec_dict.get(word)
  if embedding_vector is not None:
    embedding_matrix[i+1] = embedding_vector
# print(embedding_matrix.shape)

In [10]:
# data encoding for training
X_train_encoded = []
for i,sentence in enumerate(X_train):
    encoded_sentence = []
    for word in sentence:
        encoded_sentence.append(rank_words.get(word,0))
    X_train_encoded.append(encoded_sentence)

X_test_encoded = []
for i, sentence in enumerate(X_test):
    encoded_sentence = []
    for word in sentence:
        encoded_sentence.append(rank_words.get(word,0))
    X_test_encoded.append(encoded_sentence)


In [11]:
# data padding
X_train_pad = sequence.pad_sequences(X_train_encoded, maxlen = max_sentence_length)
X_test_pad = sequence.pad_sequences(X_test_encoded, maxlen = max_sentence_length)

X_train_sample = np.asarray(X_train_pad)
Y_train_sample = np.asarray(Y_train)
X_test_sample = np.asarray(X_test_pad)

# print(len(X_train_sample))
# print(X_train_sample.shape)
# print(len(Y_train_sample))
# print(X_test_sample.shape)

In [12]:
# create the model
model = Sequential()
model.add(Embedding(input_dim = vocabulary_size + 1, output_dim = word2Vec_embedding_size, input_length = max_sentence_length, embeddings_initializer = Constant(embedding_matrix)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
# model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
model.fit(X_train_sample, Y_train_sample, epochs = 5, batch_size = 16)

2022-03-21 18:58:51.628454: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/5
1141/1141 [==============================] - 63s 53ms/step - loss: 0.5006 - accuracy: 0.7520
Epoch 2/5
1141/1141 [==============================] - 79s 69ms/step - loss: 0.2542 - accuracy: 0.9010
Epoch 3/5
1141/1141 [==============================] - 65s 57ms/step - loss: 0.1454 - accuracy: 0.9476
Epoch 4/5
1141/1141 [==============================] - 69s 60ms/step - loss: 0.0994 - accuracy: 0.9640
Epoch 5/5
1141/1141 [==============================] - 58s 51ms/step - loss: 0.0727 - accuracy: 0.9705


### Output Prediction Result File

In [13]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []
result_probabilities = model.predict(X_test_sample)
results = [1 if probability>=0.5 else 0 for probability in result_probabilities]

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [14]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [15]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]
zero = results.count(0)
one = results.count(1)
print(zero)
print(one)

1063
965


In [42]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions2.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')